## Growth stage classification by random forest, for a chosen date ('search date') over a specified area ('extent')

1. Specify parameters , 'search date' and 'extent (i.e., polygon: lampung)

    a. the usage of 'extent' is the same as that of vegetable classification (What is extent? Gjson?)

    b. String of Date

2. Search the corresponding S1A SAR data, and download and preprocess them.

    This part is the same as that of vegetable classification. The only difference is that we only download and preprocess S1A SAR. Data that are close to the search date ('dateStr'), e.g., S1A data on'20190430' and S1A data on '20190418'

   Q1: From Data in Vegetable classification, we take 2 sample? is it correct?
   Q2: Need stack or not? base on searchdate?
   Q3: search date is random or must have sequential patern
   
   Then, we obtain, 2 file with img extension:
   
   img_current, and 
   img_previous 
   
   The code will resul variable img_current and img_previous, where each image has two channels, VH and VV, 
    
    e.g.:
    - img_current(:,:,1) refers to the VH channel, while img_current(:,:,2) refers to VV channel in current date
    - img_previous(:,:,1) refers to the VH channel, while img_preious(:,:,2) refers to VV channel in previous date
   

In [2]:
# Provide TsList
import os, datetime
from datetime import timedelta, date
from scipy.io import loadmat, savemat
import numpy as np
import pandas as pd
import time
from pathlib import Path
from PIL import Image
from spectral import *
from __future__ import print_function
import tifffile as tiff
from tqdm import tqdm
import pandas as pd
home = os.getcwd()
folder= home+r'\Validation_of_products_2018\\S1A_timeseries\\Lampung_S1A_timeseries_2018Anual_Medium.data'

In [3]:
# Built in Function

def nearest_date(items, pivot):
    return min(items, key=lambda x: abs(x - pivot))

def get_list_date(files):
    temp = files.split('.')[0].split('_')
    return datetime.datetime.strptime(temp[len(temp)-1], '%d%b%Y')

def find_files(list_date, files, ext, j, dateStr, daysInterval):
    date_curr = nearest_date(list_date[ext[j]], datetime.datetime.strptime(str(dateStr), '%d%b%Y'))
    date_prev = date_curr - datetime.timedelta(daysInterval)
    idx_curr = list_date[ext[j]].index(date_curr)
    idx_prev = list_date[ext[j]].index(date_prev)
    return files[ext[j]][idx_curr], files[ext[j]][idx_prev]

def get_img_file(img_file, hdr_file, val):
    hdr_vh = folder +'/'+hdr_file[0][val]
    img_vh = folder +'/'+img_file[0][val]
    hdr_vv = folder +'/'+hdr_file[1][val]
    img_vv = folder +'/'+img_file[1][val]
    l = [[hdr_vh, img_vh], [hdr_vv, img_vv]]
    if Path(hdr_vh).is_file() and Path(img_vh).is_file():
        if Path(hdr_vv).is_file() and Path(img_vv).is_file():
            get_info = envi.read_envi_header(hdr_vh)
            d1 = int(get_info['lines'])
            d2 = int(get_info['samples'])
            dim = (d1,d2,2)
            im_subset = np.zeros((dim))
    for ix in tqdm(range(len(l))):
        get_img = envi.open(l[ix][0])
        img_open = get_img.open_memmap(writeable = True)
        im = img_open[:d1,:d2,0]
        im_subset[:,:,ix] = im
    return im_subset

def normalize_rows(x):
    """
    function that normalizes each row of the matrix x to have unit length.

    Args:
     ``x``: A numpy matrix of shape (n, m)

    Returns:
     ``x``: The normalized (by row) numpy matrix.
    """
    return x/np.linalg.norm(x, ord=2, axis=1, keepdims=True)

In [4]:
# Params 
folder
dateStr = '27Dec2018';
ext1 = 'img'
ext2 = 'hdr'
splitStr       = '_'
daysInterval   = 12
time_formatStr = "%d-%b-%Y"
bandName = ['Sigma0_VH','Sigma0_VV']
folder= home+r'/Validation_of_products_2018/S1A_timeseries/Lampung_S1A_timeseries_2018Anual_Medium.data'
train_path=home+r'/Validation_of_products_2018/train_mat.mat'
ext = ['vh', 'vv']
numfiles = len(filter(lambda x: "Sigma0_" in x, os.listdir(folder)))/4

In [6]:
# Make Dictionary:
img = {
    'vh': [f for f in os.listdir(folder) if f.endswith('.' + 'img') and "_VH" in f], 
    'vv': [f for f in os.listdir(folder) if f.endswith('.' + 'img') and "_VV" in f]}
hdr = {
    'vh': [f for f in os.listdir(folder) if f.endswith('.' + 'hdr') and "_VH" in f],
    'vv': [f for f in os.listdir(folder) if f.endswith('.' + 'hdr') and "_VV" in f]}

date_img = {
    'vh': [get_list_date(i) for i in img[ext[0]]], 
    'vv': [get_list_date(i) for i in img[ext[1]]] }

date_hdr = {
    'vh': [get_list_date(i) for i in hdr[ext[0]]], 
    'vv': [get_list_date(i) for i in hdr[ext[1]]] }

In [7]:
img_file = [find_files(date_img, img, ext, j, 
                       dateStr, daysInterval) for j in range(len(date_img))]
hdr_file = [find_files(date_hdr, hdr, ext, j, 
                       dateStr, daysInterval) for j in range(len(date_hdr))]
img_file

[('Sigma0_VH_db_slv43_27Dec2018.img', 'Sigma0_VH_db_slv41_15Dec2018.img'),
 ('Sigma0_VV_db_slv44_27Dec2018.img', 'Sigma0_VV_db_slv42_15Dec2018.img')]

In [8]:
img_file

[('Sigma0_VH_db_slv43_27Dec2018.img', 'Sigma0_VH_db_slv41_15Dec2018.img'),
 ('Sigma0_VV_db_slv44_27Dec2018.img', 'Sigma0_VV_db_slv42_15Dec2018.img')]

In [9]:
img_current = get_img_file(img_file, hdr_file, 0) ## 0 for current, 1, previous
img_previous = get_img_file(img_file, hdr_file, 1) ## 0 for current, 1, previous

100%|██████████| 2/2 [00:02<00:00,  1.23s/it]


3. Extract features

    Here, we extract, 6 features: 
    
    - VH_img_current, 
    
    - VV_img_current,
    
    - VH_img_current/VV_img_current, 
    
    - VH_img_current-VH_img_previous,
    
    - VV_img_current-VV_img_previous, and
    
    - (VH_img_current/VV_img_current)-(VH_img_previous/VV_img_previous)
   
   then we will obtain : img_features

In [10]:
img_features = np.zeros((img_current.shape[0], img_current.shape[1], 6))

In [11]:
img_features[:,:,0] = img_current[:,:,0]
img_features[:,:,1] = img_current[:,:,1]
img_features[:,:,2] = img_current[:,:,0]/img_current[:,:,1]
img_features[:,:,3] = img_current[:,:,0]-img_previous[:,:,0]
img_features[:,:,4] = img_current[:,:,1]-img_previous[:,:,1]
img_features[:,:,5] = img_current[:,:,0]/img_current[:,:,1]-\
                      img_previous[:,:,0]/img_previous[:,:,1]
[d1,d2,d3] = img_features.shape
# where d1, d2, d3 refer to the row, colum, and feature dimension, and d3=6

In [12]:
img_features.shape

(4058, 7760, 6)

In [14]:
test_features = np.reshape(img_features,[d1*d2,d3]);
test_features = normalize_rows(test_features);

In [15]:
pd.DataFrame(test_features).head()

,0,1,2,3,4,5
0,-0.859008,-0.479902,0.098019,-0.124099,-0.082341,-0.003208
1,-0.864854,-0.478204,0.096384,-0.091513,-0.075168,-0.005875
2,-0.854256,-0.503531,0.091691,-0.030586,-0.084544,-0.014556
3,-0.865360,-0.491209,0.096381,-0.011458,-0.020876,-0.002945
4,-0.808251,-0.573741,0.072627,-0.027058,-0.106585,-0.013584


In [16]:
mat = loadmat(train_path)  # load mat-file
mdata = mat['train_mat']  # variable in mat file
mtype = mdata.dtype
ndata = {n: mdata[n][0,0] for n in mtype.names}
data_headline = ndata['feature_name']
headline = data_headline[0]

In [17]:
data_raw = ndata['train_data']
data_df = np.reshape(data_raw,[250,6]);
data_df = normalize_rows(data_df);
data_df = pd.DataFrame(data_raw)
#data_df.columns=['Sigma0_VH','Sigma0_VV','VH_minus_VV_dB','diff_1','diff_2','diff_3']

data_test = pd.DataFrame(ndata['train_label'])
#data_test.columns=['class']

train_features = data_df
train_labels = data_test

In [83]:
#from sklearn.datasets.samples_generator import make_blobs

# Create random data train and labels 120x5, with n labels =3
#X, y = make_blobs(n_samples=120, centers=3, n_features=6,random_state=1)
#train_features = X
#train_labels = y

#normalize train data
#train_features = normalize_rows(train_features);

#pd.DataFrame(train_features).head()
#train_labels

In [18]:
# Instantiate model with 1000 decision trees
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# Train the model on training data
rf.fit(train_features, train_labels.values.ravel());
rf.score(train_features, train_labels.values.ravel())

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


0.8905615464285714

In [19]:
# Use the forest's predict method on the test data
predictions = rf.predict(test_features)

In [110]:
# Predict result
pd.DataFrame(predictions).head()

,0
0,2.003
1,2.003
2,2.005
3,2.005
4,2.000


In [86]:
from sklearn import metrics

#print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
#print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
#print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))  

4. Train random forest classifier, and do classification

   You can use 'sklearn.ensemble.RandomForestClassifier'

In [111]:
from sklearn.ensemble import RandomForestClassifier

In [87]:
# e.g. Training Data set
import numpy as np
import pandas as pd

num_cls = 4;
num_obs_each_cls = 30;

train_labels = np.zeros((4*30,1)); 
train_features = np.zeros((4*30,6));

In [88]:
#from sklearn.datasets import make_classification

#X, y = make_classification(n_samples=120, n_features=6, n_informative=3, n_classes=3,
#                           n_redundant=0, random_state=0, shuffle=False)

In [161]:
from sklearn.model_selection import train_test_split

X = data_df
y = data_test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)

In [162]:
rfc =  RandomForestClassifier(n_estimators = 1000, random_state = 42, max_depth=15)
rfc.fit(X_train, y_train.values.ravel());
rfc.score(X_train, y_train.values.ravel());

In [163]:
y_pred = rfc.predict(X_test)

In [164]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred))  
print(accuracy_score(y_test, y_pred))  

[[ 3  0  3  0]
 [ 2  0  0  0]
 [ 2  0 10  0]
 [ 0  0  1  4]]
             precision    recall  f1-score   support

          1       0.43      0.50      0.46         6
          2       0.00      0.00      0.00         2
          3       0.71      0.83      0.77        12
          4       1.00      0.80      0.89         5

avg / total       0.65      0.68      0.66        25

0.68


In [118]:
# Use the forest's predict method on the test data
predictions_clf = rfc.predict(test_features)

In [156]:
df_predict = pd.DataFrame(predictions_clf)
df_predict.columns = ['label']

In [1]:
y_pred

NameError: name 'y_pred' is not defined